The goal is to create a portfolio with a perfect hedge and see how the EAD is distributed.
We load two perfectly offsetting IRS (one payer, one receiver). Just to not stumble over small epsilons we add an equity option to put it all a bit into proportion.

In [1]:
import QuantLib as ql
from allocation.Enums import FdApproach2
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.irs import IRS
from marketdata import init_marketdata

In [2]:
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection
IRS_pay = IRS(notional=100000000,
              timeToSwapStart=ql.Period(2, ql.Days),
              timeToSwapEnd=ql.Period(10, ql.Years),
              swapDirection=SwapDirection.PAYER,
              index=InterestRateIndex.USDLIBOR3M)

IRS_rec = IRS(notional=100000000,
              timeToSwapStart=ql.Period(2, ql.Days),
              timeToSwapEnd=ql.Period(10, ql.Years),
              swapDirection=SwapDirection.RECEIVER,
              index=InterestRateIndex.USDLIBOR3M)

eqOpt = EquityOption(notional = 1000000)

ca = CollateralAgreement()
ca.link_sa_ccr_instance(SA_CCR(ca))

In [3]:
ca.add_trades([IRS_pay, IRS_rec, eqOpt])
print(ca.get_sa_ccr_model().get_risk_measure())
print(ca.get_im_model().get_risk_measure())

1319277.1719911932
6952717.387110085


In [4]:

eulerAllocator = EulerAllocator(ca)
im_alloc_forward = eulerAllocator.allocate_im()
saccr_alloc_forward = eulerAllocator.allocate_ead()

eulerAllocator.fdApproach2=FdApproach2.Central
im_alloc_central = eulerAllocator.allocate_im()
saccr_alloc_central = eulerAllocator.allocate_ead()

eulerAllocator.fdApproach2=FdApproach2.Backward
im_alloc_backward = eulerAllocator.allocate_im()
saccr_alloc_backward = eulerAllocator.allocate_ead()

In [5]:
EAD_IRSpay_forward = saccr_alloc_forward[IRS_pay]
EAD_IRSrec_forward = saccr_alloc_forward[IRS_rec]
EAD_EqOpt_forward = saccr_alloc_forward[eqOpt]
EAD_forward_sum = EAD_EqOpt_forward+EAD_IRSpay_forward+EAD_IRSrec_forward
Delta_EAD_forward = ca.get_sa_ccr_model().get_risk_measure() - EAD_forward_sum

EAD_IRSpay_central = saccr_alloc_central[IRS_pay]
EAD_IRSrec_central = saccr_alloc_central[IRS_rec]
EAD_EqOpt_central = saccr_alloc_central[eqOpt]
EAD_central_sum = EAD_EqOpt_central+EAD_IRSpay_central+EAD_IRSrec_central
Delta_EAD_central = ca.get_sa_ccr_model().get_risk_measure() - EAD_central_sum

EAD_IRSpay_backward = saccr_alloc_backward[IRS_pay]
EAD_IRSrec_backward = saccr_alloc_backward[IRS_rec]
EAD_EqOpt_backward = saccr_alloc_backward[eqOpt]
EAD_backward_sum = EAD_EqOpt_backward+EAD_IRSpay_backward+EAD_IRSrec_backward
Delta_EAD_backward = ca.get_sa_ccr_model().get_risk_measure() - EAD_backward_sum

IM_IRSpay_forward = im_alloc_forward[IRS_pay]
IM_IRSrec_forward = im_alloc_forward[IRS_rec]
IM_EqOpt_forward = im_alloc_forward[eqOpt]
IM_forward_sum = IM_EqOpt_forward+IM_IRSpay_forward+IM_IRSrec_forward
Delta_IM_forward = ca.get_sa_ccr_model().get_risk_measure() - IM_forward_sum

IM_IRSpay_central = im_alloc_central[IRS_pay]
IM_IRSrec_central = im_alloc_central[IRS_rec]
IM_EqOpt_central = im_alloc_central[eqOpt]
IM_central_sum = IM_EqOpt_central+IM_IRSpay_central+IM_IRSrec_central
Delta_IM_central = ca.get_sa_ccr_model().get_risk_measure() - IM_central_sum

IM_IRSpay_backward = im_alloc_backward[IRS_pay]
IM_IRSrec_backward = im_alloc_backward[IRS_rec]
IM_EqOpt_backward = im_alloc_backward[eqOpt]
IM_backward_sum = IM_EqOpt_backward+IM_IRSpay_backward+IM_IRSrec_backward
Delta_IM_backward = ca.get_im_model().get_risk_measure() - IM_backward_sum

pf_ead = ca.get_sa_ccr_model().get_risk_measure()
pf_im = ca.get_im_model().get_risk_measure()

For reference a not perfectly hedged portfolio

In [6]:
IRS_pay = IRS(notional=200000000,
              timeToSwapStart=ql.Period(2, ql.Days),
              timeToSwapEnd=ql.Period(10, ql.Years),
              swapDirection=SwapDirection.PAYER,
              index=InterestRateIndex.USDLIBOR3M)

IRS_rec = IRS(notional=100000000,
              timeToSwapStart=ql.Period(2, ql.Days),
              timeToSwapEnd=ql.Period(10, ql.Years),
              swapDirection=SwapDirection.RECEIVER,
              index=InterestRateIndex.USDLIBOR3M)

eqOpt = EquityOption(notional = 1000000)

ca = CollateralAgreement()
ca.link_sa_ccr_instance(SA_CCR(ca))

ca.add_trades([IRS_rec, IRS_pay, eqOpt])

eulerAllocator = EulerAllocator(ca)
ead_alloc = eulerAllocator.allocate_ead()
im_alloc = eulerAllocator.allocate_im()

ead = ca.get_sa_ccr_model().get_risk_measure()
im = ca.get_im_model().get_risk_measure()

sum_ead_alloc = sum([v for v in ead_alloc.values()])
sum_im_alloc = sum([v for v in im_alloc.values()])